传统的Seq2Seq模型包含两部分，一部分叫编码器，另一部分叫解码器。
把原序列送入编码器之中，得到固定的向量表征，通常是编码器的最后一个隐藏状态作为整个原序列固定的表征，解码器就基于这个固定的表征来生成目标序列。

它利用编码器得到的是一个固定大小的表征向量，这里就隐含一个假设，它假设编码器能够将输入序列所有的信息压缩到这个固定长度的向量中，那对于较长的序列，显然是很困难的，即这个假设很难成立。

为了解决这个问题，15年有人提出了基于attention机制的seq2seq模型。编码器不再是输出一个固定大小的向量了，而是输出与输入序列等长的一组向量，然后decoder也不再使用固定的向量作为输入了，而是每次生成一个预测值的时候，都会去编码器生成的这一组向量中挑选相关度最高的向量作为输入。 

每次预测，都会通过上一个预测输出和编码器输出的一组向量生成 一个新的隐藏状态 c_i 作为当前lstm单元的输入：

$$c_i = \sum_{j=1}^{T_x} a_{ij}h_j$$
$h_{j:T_x}$为编码器生成的一组向量，$a_{ij}$为注意力分布,为第i时刻对整个编码器输出向量h_j的注意力分数，计算方式如下：
$$a_{ij} = \frac{\exp(<s_{i-1}, h_j>)}{\sum_{k=1}^{T_x}\exp(<s_{i-1}, h_k>)}$$


分为训练阶段和预测阶段:
+ 预测的时候，每个lstm单元的输入包括上一个时刻的输出 和  注意力机制得到的context
+ 训练的时候，每个lstm单元的输入包括上一个时刻的真实值 和 注意力机制得到的context

context是上个时刻的输出状态和编码器状态计算得到

核心在于attention机制,计算的输入包括两部分：
+ 解码器$t$时刻的状态输出: `decoder_state_t: [1, batch_size, hidden_size]`
+ 编码器的完整状态输出: `endoder_states: [batch_size, src_seq_len, hidden_szie]`

计算的思路是先把 decoder_state_t 扩展为 和 endoder_states 维度相同的，然后于 endoder_states 逐元素相乘，再在dim=1维度求和，得到[batch_size, src_seq_len] 的矩阵，表示相似度矩阵。就可以再 dim=1 维度 进行softmax 操作。


把 `decoder_state_t` 扩展为 和 `endoder_states` 维度相同的张量后，第二个维度表示的就是时间维度，只不过 `decoder_state_t` 每个时间的向量都是一样的。然后和 `endoder_states` 逐元素相乘，再在时间维度求和，其实就相当于 `decoder_state_t` 和 `endoder_states` 每个时刻的向量都做了内积。

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [2]:
class Seq2SeqEncoder(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(Seq2SeqEncoder, self).__init__()

        self.lstm_layer = nn.LSTM(input_size, hidden_size, batch_first=True)

    def forward(self, input_seq):
        """
        params:
            input_seq:[batch_size, seq_len, input_size]
        return:
            outputs: [batch_size, seq_len, hidden_size]，只包含状态 h
            final_h: [1, batch_size, hidden_size]
            final_c: [1, batch_size, hidden_size]
        """
        outputs, (final_h, final_c) = self.lstm_layer(input_seq)
        return outputs, (final_h, final_c)

In [6]:
# test
input_seq = torch.randn(128, 12, 3)
encoder = Seq2SeqEncoder(3, 7)
outputs, (final_h, final_c) = encoder(input_seq)
print(outputs.shape, final_h.shape, final_c.shape)

torch.Size([128, 12, 7]) torch.Size([1, 128, 7]) torch.Size([1, 128, 7])


In [3]:
class Seq2SeqAttentionMechanism(nn.Module):
    def __init__(self):
        super(Seq2SeqAttentionMechanism, self).__init__()

    def forward(self, decoder_state_t, encoder_states):
        """
        param:
            decoder_state_t 解码器某一时刻的状态输出 [batch_size, hidden_size]
            encoder_states: 编码器输出的各个状态 [batch_size, src_seq_len, hidden_size]
        return:
            context: 计算得到的加权求和 [batch_szie, hidden_size]
            atten_prob:注意力分布 [batch_size, src_seq_len]
        """
        batch_size, src_seq_len, hidden_size = encoder_states.shape

        decoder_state_t = decoder_state_t.unsqueeze(dim=1) # 转为 batch_size, 1, hidden_size
        decoder_state_t = torch.tile(decoder_state_t, dims=(1, src_seq_len, 1)) # 转为 batch_size, src_sen, hiddensize
        # 计算 注意力分数
        score = torch.sum(decoder_state_t * encoder_states, dim=-1) # [batch_size, src_seq_len]

        atten_prob = torch.softmax(score, dim=1) # [batch_size, src_seq_len]
        atten_prob = atten_prob.unsqueeze(-1) # [batch_size, src_seq_len, 1]
        # 加权求和 :逐元素相乘，再在 序列长度维度求和
        context = torch.sum(atten_prob * encoder_states, dim=1)  [batch_size, hidden_size]

        return context, atten_prob.squeeze(-1)

In [7]:
class Seq2SeqDecoder(nn.Module):
    def __init__(self, input_size, hidden_size):
        # 因为每一步都要计算注意力分数，生成 新的状态，所以不能直接采用LSTM

        self.lstm_cell = nn.LSTMCell(input_size + hidden_size, hidden_size)
        self.attention_mechanism = Seq2SeqAttentionMechanism()

    def forward(self, input_seq, target_seq, encoder_states, encoder_final_h_c):
        """
        params:
            input_seq: [batch_size, src_seq_len, input_size ] 需要用最后一个时刻的值作为 解码器的初始
            input_seq: [batch_size, tgt_seq_len, input_size ] 训练的时候是使用上一时刻真实值作为当前输入
            encoder_states: [batch_size, src_seq_len, hidden_size] 编码器得到的向量
            encoder_final_h_c:(h_t, c_t), [1, batch_size, hidden_size] 用做解码器的初始状态
        return:
            pred_outputs: [batch_size, tgt_seq_len, hidden_size]
        """
        batch_size, tgt_seq_len, hidden_size = target_seq.shape
        init_input_0 = input_seq[:, 0, :]
        # 初始的 h c
        h_t, c_t = encoder_final_h_c


        pred_outpus = torch.zeros(size=(batch_size, tgt_seq_len, hidden_size))
        for t in range(tgt_seq_len):
            if t == 0:
                decoder_input_t = input_seq[:, -1, :]
            else:
                decoder_input_t = target_seq[:, t-1, :]  # 使用真实值作为解码器输入
            # 根据上一时刻的输出状态，计算新的输入
            context, _ = self.attention_mechanism(h_t, encoder_states) # [batch_size, hidden_size]

            decoder_input_t = torch.cat([decoder_input_t, context], dim=1) # [batch_size, input_size + hidden_size]

            h_t, c_t = self.lstm_cell(decoder_input_t, (h_t, c_t))

            # 将预测保存下来
            pred_outpus[:, t, :] = h_t

        return pred_outpus

    def inference(self, input_seq, target_seq_len, encoder_states, encoder_final_h_c):
        """
        params:
            input_seq: [batch_size, src_seq_len, input_size ] 需要用最后一个时刻的值作为 解码器的初始
            target_seq_len: 预测序列的长度
            encoder_states: [batch_size, src_seq_len, hidden_size] 编码器得到的向量
            encoder_final_h_c:(h_t, c_t), [1, batch_size, hidden_size] 用做解码器的初始状态
        return:
            pred_outputs: [batch_size, tgt_seq_len, hidden_size]
        """
        # 初始的 h c
        h_t, c_t = encoder_final_h_c

        batch_size, _, hidden_size = input_seq.shape
        pred_outpus = torch.zeros(size=(batch_size, target_seq_len, hidden_size))
        for t in range(target_seq_len):
            if t == 0:
                decoder_input_t = input_seq[:, -1, :]
            else:
                decoder_input_t = pred_outpus[:, t-1, :]  # 使用上一个输出作为 结果
            # 根据上一时刻的输出状态，计算新的输入
            context, _ = self.attention_mechanism(h_t, encoder_states) # [batch_size, hidden_size]
            decoder_input_t = torch.cat([decoder_input_t, context], dim=1) # [batch_size, input_size + hidden_size]

            # 计算当前时刻的状态
            h_t, c_t = self.lstm_cell(decoder_input_t, (h_t, c_t))
                
            # 将预测保存下来
            pred_outpus[:, t, :] = h_t

        return pred_outpus

In [8]:
class Seq2Seq(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(Seq2Seq, self).__init__()
        self.encoder = Seq2SeqEncoder(input_size, hidden_size)
        self.decoder = Seq2SeqDecoder(input_size, hidden_size)

    def forward(self, input_seq, target_seq):
        encoder_states, (final_h, final_c) = self.encoder(input_seq)
        preds_outputs = self.decoder(input_seq, target_seq, encoder_states, (final_h, final_c))
        return preds_outputs

    def inference(self, input_seq, target_seq_len):
        encoder_states, (final_h, final_c) = self.encoder(input_seq)
        # 预测的时候是要使用 inference 函数
        preds_outputs = self.decoder.inference(input_seq, target_seq_len, encoder_states, (final_h, final_c))
        return preds_outputs